In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score
import time
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from imblearn import over_sampling
import pickle

C:\Users\Adelia\AppData\Local\Temp\ipykernel_21808\3878294190.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
def eval_classification(model, avg = 'binary'):
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    y_pred_proba = model.predict_proba(X_test)
    y_pred_proba_train = model.predict_proba(X_train)
    
    print("Accuracy (Test Set): %.6f" % accuracy_score(y_test, y_pred))
    print("Accuracy (Train Set): %.6f" % accuracy_score(y_train, y_pred_train))
    print("Precision (Test Set): %.6f" % precision_score(y_test, y_pred, average = avg))
    print("Precision (Train Set): %.6f" % precision_score(y_train, y_pred_train, average = avg))
    print("Recall (Train Set): %.6f" % recall_score(y_train, y_pred_train, average = avg))
    print("Recall (Test Set): %.6f" % recall_score(y_test, y_pred, average = avg))
    print("F1-Score (Train Set): %.6f" % f1_score(y_train, y_pred_train, average = avg))
    print("F1-Score (Test Set): %.6f" % f1_score(y_test, y_pred, average = avg))

    #score = cross_validate(model, X_over, y_over, cv=5, scoring='roc_auc', return_train_score=True)
    #print('roc_auc (crossval train): '+ str(score['train_score'].mean()))
    #print('roc_auc (crossval test): '+ str(score['test_score'].mean()))
def show_feature_importance(model):
    feat_importances = pd.Series(model.feature_importances_, index=X.columns)
    ax = feat_importances.nlargest(25).plot(kind='barh', figsize=(10, 8))
    ax.invert_yaxis()

    plt.xlabel('score')
    plt.ylabel('feature')
    plt.title('feature importance score')

def show_best_hyperparameter(model):
    print(model.best_estimator_.get_params())

In [3]:
df = pd.read_excel("sampling.xlsx")
df['id_model'] = range(df.shape[0])

In [4]:
data = df.drop(columns = ['cleaned_poi_name','cleaned_nama_merchant','similarity',
#                           'tfidf_similarity', 'levenshtein_similarity'
                         ])
data = data.sample(350).copy()

X = data.fillna(0).drop(columns = ['id_model','target'])
y = data.fillna(0)['target']

X_over, y_over = over_sampling.SMOTE().fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=47)

In [8]:
# decision tree
from sklearn.tree import DecisionTreeClassifier # import decision tree dari sklearn
dt = DecisionTreeClassifier(random_state=42) # inisiasi object dengan nama dt
dt.fit(X_train, y_train) # fit model decision tree dari data train
eval_classification(dt, avg = 'micro')

Accuracy (Test Set): 0.866667
Accuracy (Train Set): 1.000000
Precision (Test Set): 0.866667
Precision (Train Set): 1.000000
Recall (Train Set): 1.000000
Recall (Test Set): 0.866667
F1-Score (Train Set): 1.000000
F1-Score (Test Set): 0.866667


In [53]:
#randomforest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
eval_classification(rf, avg = 'micro')


Accuracy (Test Set): 0.962
Accuracy (Train Set): 1.000
Precision (Test Set): 0.962
Precision (Train Set): 1.000
Recall (Train Set): 1.000
Recall (Test Set): 0.962
F1-Score (Train Set): 1.000
F1-Score (Test Set): 0.962


In [54]:
y_pred = rf.predict(X_test)
y_pred_train = rf.predict(X_train)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, y_pred_train)
pd.DataFrame(cm)

,0,1
0,92,0
1,0,153


In [24]:
#hyperparameter tuning Randomforest
"""
    n_estimators=100,
    *,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='sqrt',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None,
    ccp_alpha=0.0,
    max_samples=None,
"""
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np

#Menjadikan ke dalam bentuk dictionary
hyperparameters = {
                    'n_estimators' : [int(x) for x in np.linspace(10, 30, num = 10)],
                    'criterion':["gini", "entropy", "log_loss"],
                    'max_depth' : [int(x) for x in np.linspace(8, 15, num = 50)],
                    'min_samples_split' : [int(x) for x in np.linspace(8, 20, num = 9)],
                    'min_samples_leaf':[int(x) for x in np.linspace(10, 30, num = 9)],
            
                    }

# Init
rf = RandomForestClassifier(random_state=42)
rf_tuned = RandomizedSearchCV(rf, hyperparameters, cv=5, random_state=42, scoring='roc_auc')
rf_tuned.fit(X_train,y_train)

# Predict & Evaluation
eval_classification(rf_tuned, avg = 'weighted')

Accuracy (Test Set): 0.933
Accuracy (Train Set): 0.904
Precision (Test Set): 0.933
Precision (Train Set): 0.904
Recall (Train Set): 0.904
Recall (Test Set): 0.933
F1-Score (Train Set): 0.904
F1-Score (Test Set): 0.933


In [9]:
y_pred = rf_tuned.predict(X_test)
y_pred_train = rf_tuned.predict(X_train)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, y_pred_train)
pd.DataFrame(cm)

,0,1
0,86,9
1,12,173


In [26]:
pickle.dump(rf_tuned, open('model_v1.pkl', 'wb'))

In [41]:
pickled_model = pickle.load(open('model_v1.pkl', 'rb'))

In [56]:
# test
df_test = df[~df['id_model'].isin(data['id_model'].tolist())]
# df_test = df.sample(100)
X_test = df_test[X.columns.tolist()].fillna(0)

In [57]:
predict = pickled_model.predict(X_test)
df_test['target_model'] = predict

C:\Users\Adelia\AppData\Local\Temp\ipykernel_4752\2371292965.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['target_model'] = predict


In [58]:
target = df_test['target'].tolist()
target_model = df_test['target_model'].tolist()
cm = confusion_matrix(target, target_model)
# Extract TN, FP, FN, TP
TN, FP, FN, TP = cm.ravel()

# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Display the results
print(f'True Positives (TP): {TP}')
print(f'True Negatives (TN): {TN}')
print(f'False Positives (FP): {FP}')
print(f'False Negatives (FN): {FN}')
print(f'Accuracy: {accuracy:.2f}')

True Positives (TP): 87
True Negatives (TN): 58
False Positives (FP): 3
False Negatives (FN): 2
Accuracy: 0.97
